In [1]:
import numpy as np
import pptk
import networkx as nx
import imageio
import glob
import re
from random import sample, seed
from scipy.spatial import distance
from time import strftime
import os
import scipy.optimize as optimize
import math
from sklearn.linear_model import LinearRegression
import time
import copy
from moving_least_square import *
from tempfile import TemporaryFile
from collections import defaultdict
from vpython import *
from math import *
scene = canvas()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
seed(20)
if os.name == 'nt': # Windows
    system_win = 1
else:
    system_win = 0

In [3]:
# Given the data points, return the MST of the point cloud generated from the PNG files and an array 
# of points positions
# Param: data: an array of points coordinate. Each point's coordinate should has the format of [x, y, z]
#        drawMST: boolean value. Default is false. If set true, the function will also draw a MST graph at the end
#        sampleNumber: int value. Default is 5000. This function will only sample <sampleNumber> points from the data
# Return: a NetworkX graph representing the Minimum spanning tree of the data points
def getMSTFromDataPoint(data, drawMST: bool=False, sampleNumber: int=5000):
    # Read points data from PNGs 
    if(sampleNumber > len(data)):
        sampleNumber = len(data)
        
    # default sample 5000 points from the whole set, otherwise it would take too long
    print("---------------")
    print("There are " + str(len(data)) + " points in total. Now sampleling " + str(sampleNumber) + " points from them")
    sample_data = np.asarray(sample(list(data), sampleNumber))
    print("---------------")
    print("Done!")
    
    #display the points 
    #displayPoints(sample_data, 1.3)
    
    #Create a networkX graph instance that represent MST
    print("---------------")
    print("Begin creating a MST of the sampled points cloud")
    MST = CreateMSTGraph(sample_data)
    print("---------------")
    print("MST creation Done!")
    
    if(drawMST):
        nx.draw(MST, dict(enumerate(sample_data[:, :2])))
        
    return (MST, sample_data)
    

In [4]:
# This function will invoke a pptk viewer to render the points 
# Param: data: an array of points coordinate. Each point's coordinate should has the format of [x, y, z]
#       pointSize: the size of the point to be rendered on the screen
# Return: none

def displayPoints(data, pointSize):
    v = pptk.viewer(data)
    v.set(point_size=pointSize)

In [5]:
# This function will read a txt file and convert its content to point data, which is an array of points coordinate. 
# Each point's coordinate should has the format of [x, y, z]
# Param: filePath: the file path of the txt file
# Return: an array of points coordinate. Each point's coordinate has the format of [x, y, z]
def readPointFromTXT(filePath):
    data = np.genfromtxt(fname=filePath, skip_header=0)
    return data

In [6]:
# This function will read a series of PNG file and convert its content to point data, which is an array of points coordinate. 
# Each point's coordinate will have the format of [x, y, z]
# Param: filePath: the file path of the PNG files. Each file should be named as 1.png, 2.png, 3.png ... etc. All the png file should 
# be ordered by the their topological order from their original dicom file
# Return: an array of points coordinate. Each point's coordinate has the format of [x, y, z]
def ReadPointFromPNG(filepath):
    print("---------------")
    print("Begin reading points data from PNG files")
    path_list = [im_path for im_path in glob.glob(filepath)]
    
    if system_win:
        path_list_parsed = [re.split('\\\\|\.', path) for path in path_list]
    else:
        path_list_parsed = [re.split('/|\.', path) for path in path_list]
    path_list_parsed_valid = [x for x in path_list_parsed if x[-1] == 'png']
    path_list_parsed_valid = sorted(path_list_parsed_valid, key=lambda x:int(x[-2]))
    
    print("There are", len(path_list_parsed_valid),"PNG files, now convert them to point coordinates in 3D")
    imageData = []
    delta = 0.5
    totalThickness = len(path_list_parsed_valid) * 3 * delta
    for path in path_list_parsed_valid:
        s = ""
        if system_win:
            s = "\\"
        else:
            s = "/"
        s = s.join(path)
        s = s[:-4] + '.png'
        image = imageio.imread(s)
        imageData.append(image)
    
    zxy = np.transpose(np.where(imageData))
    xyz = zxy[:, [1, 2, 0]]
    xyz[:, 2] = totalThickness - xyz[:, 2]*3*delta
    print("Done!")
    return xyz

In [7]:
# This function is used to limited the number of edges in the original graph.
# Instead of creating a graph with full connectivity, this function will return 
# a list of neighbor points for each point and we will only connect them in the graph
def getNearbyPoints(pointsData):
    D = distance.squareform(distance.pdist(pointsData))
    closestIndicies = np.argsort(D, axis=1)
    closestDis = np.sort(D, 1)
    threshold = 10 # This number can be changed. The greater this number, the more edges
    return (closestIndicies[:, 1:threshold], closestDis[:, 1:threshold])

In [8]:
def CreateMSTGraph(pointsData):
    print("---------------")
    print("Begin calculating nearby points for each point")
    nearbyInfo = getNearbyPoints(pointsData)
    print("---------------")
    print("Nearby points calculation Done!")
    print("---------------")
    print("Begin construct graph")
    G=nx.Graph()
    closestIndicies = nearbyInfo[0]
    closestDis = nearbyInfo[1]
    for firstPIndex in range(len(closestIndicies)):
        for second in range(len(closestIndicies[firstPIndex])):
            secondPIndex = closestIndicies[firstPIndex][second]
            G.add_edge(firstPIndex, secondPIndex , weight = closestDis[firstPIndex][second])
    print("---------------")
    print("Graph construction Done!")
    print("---------------")
    print("Begin calculate MST")
    G = nx.minimum_spanning_tree(G)
    print("---------------")
    print("MST calculation Done!")
    return G

In [9]:
# Impliment the collect algorithm for 3D points in the paper
def collectPoints1(P: int, PStar: int):
    global H_glo
    global graph
    global pointsCor3D
    global A

    A.append(P)
    for edge in graph.edges(P):
        Pj = edge[1]
        if(Pj) not in A and distance.euclidean(pointsCor3D[Pj], pointsCor3D[PStar]) < H_glo:
            collectPoints1(Pj, PStar)

In [10]:
# This function will collect the neighbors of PStar and return a list of this points's index
# Param: PStar: the index of the point that we want to find its neighbors
# This function will also maintain the dictionary of the distance between points and the weight 
# between points. 
def collectPointsNonrec(PStar: int, H:int):
    
    global graph
    global pointsCor3D
    global distance_dict 
    global dirty_dict
    
    toExplore = [PStar]
    A = [PStar]
    distance_dict[(PStar, PStar)] = 0
    weight_dict[((PStar, PStar))] = 1
    
    while len(toExplore) > 0:
        curP = toExplore[0]
        del toExplore[0]
        for Pj in graph.neighbors(curP):
            if(Pj) not in A:
                
                # Maintain the dictionary of distance and weight between points
                if (Pj, PStar) not in distance_dict or (PStar, Pj) not in distance_dict or \
                dirty_dict[PStar] == 1 or dirty_dict[Pj] == 1:
                    dist_temp = distance.euclidean(pointsCor3D[Pj], pointsCor3D[PStar])
                    distance_dict[(Pj, PStar)] = dist_temp
                    distance_dict[(PStar, Pj)] = dist_temp
                    weight_dict[((PStar, Pj))] = weightFun(PStar, Pj)
                    weight_dict[((Pj, PStar))] = weightFun(PStar, Pj)
                    dirty_dict[Pj] = 0
                    dirty_dict[PStar] = 0
                    
                if distance_dict[(Pj, PStar)] < H:
                    toExplore.append(Pj)
                    A.append(Pj)
    return A

In [11]:
def weightFun(P1, P2):
    global distance_dict 
    global dirty_dict
    global pointsCor3D
    if(P1 == P2):
        return 1
    return math.exp(-1 * (distance_dict[(P1, P2)]**2)/(H_glo**2))

In [12]:
def calculateRegressionPlane(PStar, A: list):
    global pointsCor3D
    global H_glo
    global weight_dict
    global curPlaneGuess
    #global curScalar
    #global curALen
    '''
    weightKeyList = [(PStar, x) for x in A[curALen:]]
    wM = np.array([weight_dict[k] for k in weightKeyList])
    
    xMatrix = np.array([pointsCor3D[point][0] for point in A[curALen:]])
    yMatrix = np.array([pointsCor3D[point][1] for point in A[curALen:]])
    zMatrix = np.array([pointsCor3D[point][2] for point in A[curALen:]])
    scalarList = np.array([np.sum(xMatrix**2*wM), 2*np.sum(xMatrix*yMatrix*wM), 2*np.sum(xMatrix*wM), \
                           -2*np.sum(xMatrix*zMatrix*wM), np.sum(yMatrix**2*wM), 2*np.sum(yMatrix*wM), \
                           -2*np.sum(yMatrix*zMatrix*wM), np.sum(wM) ,-2*np.sum(zMatrix*wM), \
                           np.sum(zMatrix**2*wM)]) + curScalar
    '''
    
    weightKeyList = [(PStar, x) for x in A]
    wM = np.array([weight_dict[k] for k in weightKeyList])
    
    xMatrix = np.array([pointsCor3D[point][0] for point in A])
    yMatrix = np.array([pointsCor3D[point][1] for point in A])
    zMatrix = np.array([pointsCor3D[point][2] for point in A])
    
    def f(params):
        a, b, c = params 
        loss = sum(((a * xMatrix + b*yMatrix + c - zMatrix)**2)*wM)
        #loss = a**2*scalarList[0] +  a*b*scalarList[1] + a*c*scalarList[2] + a*scalarList[3] + b**2*scalarList[4]\
        #+ b*c*scalarList[5] + b*scalarList[6] + c**2*scalarList[7] + c*scalarList[8] + scalarList[9]
        
        '''
        for point in A:
            point_cor = pointsCor3D[point]
            loss += ((a*point_cor[0] + b*point_cor[1] + c - point_cor[2])**2)*weight_dict[((point, PStar))]
            #loss += ((a*point_cor[0] + b*point_cor[1] + c - point_cor[2])**2)*weightFun(PStar, point)
        '''
        return loss
    
    result = optimize.minimize(f, curPlaneGuess, method = 'Nelder-Mead')
    
    if result.success:
        fitted_params = result.x
    else:
        raise ValueError(result.message)
        
    curPlaneGuess = fitted_params
    #curScalar = scalarList
    #curALen = len(A)
    
    return fitted_params

In [13]:
def projectPoints(params, A: list):
    global pointsCor3D
    a, b, c = params
    normal = np.asarray([a, b, -1])
    normal = normal / np.linalg.norm(normal)
    pointOnPlane = np.asarray([0, 0, c])
    projectionPointsCor = []
    for point in A:
        point_cor = np.asarray(pointsCor3D[point])
        pointToPlaneV = point_cor - pointOnPlane
        dist = np.dot(normal, pointToPlaneV)
        projectionPointcor = point_cor - dist*normal
        projectionPointsCor.append(list(projectionPointcor))
    return projectionPointsCor

In [14]:
# this function converted the 3D coordinate system of points in a plane to 2D, returns a list of new coordinates
# each of them also has x, y and z component but z is equal to 0
# this finction also will return the info of the plane, which can be used to convert a 2D coordinate to 3D again
# The format of the plane info is [u, v, origin] (u is a unit vector in 3D representing plane's x axis, y is a unit 
# vector in 3D representing plane's y axis, origin is a coordinate in 3D of plane's origin )
def convertTo2DCor(pointsCor, planeParam):
    a, b, c = planeParam
    origin = np.array([0, 0, c])
    u = np.array([0, 0, c]) - np.array([1, 1, a + b + c])
    u = u / np.linalg.norm(u)
    normal = np.array([a, b, -1])
    v = np.cross(u, normal)
    v = v / np.linalg.norm(v)
    convertedPointsCor = []
    
    for pointCor in pointsCor:
        oriV = np.array(pointCor) - origin
        new_x = np.dot(oriV, u)
        new_y = np.dot(oriV, v)
        convertedPointsCor.append([new_x, new_y, 0])
        
    planeInfo = [u, v, origin]
    
    return (convertedPointsCor, planeInfo)
    

In [15]:
# Param: targetPoint: the index of the point that we want to find its neighbors and their coordinate in 2D 
# Return: the 2D coordinate of the 3D points and the information of the regression plane, which the points are located
# require gloable perameters graph and pointsCor3D

def get2DCorFrom3D(targetPoint):
    
    global graph
    global pointsCor3D
    global H_glo
    global H_delta
    global min_neighbors
    
    localPoints = []

    while (len(localPoints) < min_neighbors):
        localPoints = collectPointsNonrec(targetPoint, H_glo)
        H_glo += H_delta

    params = calculateRegressionPlane(targetPoint, localPoints)

    projectionPointsCor = projectPoints(params, localPoints)

    points2DCor, planeInfo = convertTo2DCor(projectionPointsCor, params)

    return (points2DCor, planeInfo)

In [16]:
# This function takes a single point's 2D coordinate and transform it into 3D base on the planeInfo
def get3DCorFrom2D(pointCor, planeInfo):
    u, v, origin = planeInfo
    vectorElem1 = pointCor[0]*u
    vectorElem2 = pointCor[1]*v
    newCor = vectorElem1 + vectorElem2 + origin
    
    return newCor

In [17]:
#compue the line regression
def calculateRegressionLine(pointsCor):
    X = np.array([x[0] for x in pointsCor]).reshape(-1, 1)
    Y = np.array([x[1] for x in pointsCor]).reshape(-1, 1)
    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    
    return(linear_regressor.coef_[0], linear_regressor.intercept_[0])

In [18]:
def rotatePointsCor(pointsCor, lineCoef):
    pointsCor = np.array(pointsCor)
    theta = math.atan(lineCoef)
    c, s = math.cos(theta), math.sin(theta)
    R = np.array([(c,-s, 1), (s, c, 1)])
    newPointsCor = []
    
    for point in pointsCor:
        newPointsCor.append(R.dot(point))
    return np.asarray(newPointsCor)

In [19]:
def deleteChild(child:int):
    global removedNodeDict
    global graph_centerline
    global pointsCor3D_centerline
    
    graph_centerline.remove_node(child)
    
    for grandChild in removedNodeDict[child]:
        if(graph_centerline.has_node(grandChild)):
            deleteChild(grandChild)
        

In [20]:
def addBackChildren(parent:int, curDepth:int):
    global removedNodeDict
    global graph_centerline
    global pointsCor3D_centerline
    
    if(parent not in removedNodeDict):
        return curDepth
    
    if(len(removedNodeDict[parent]) == 1):
        child = removedNodeDict[parent][0]
        parent_cor = pointsCor3D_centerline[parent]
        child_cor = pointsCor3D_centerline[child]
        graph_centerline.add_edge(parent, child , weight=distance.euclidean(parent_cor, child_cor))
        return addBackChildren(child, curDepth + 1)
    
    else:
        maxDepth = 0
        curChild = -1
        
        for child in removedNodeDict[parent]:
            parent_cor = pointsCor3D_centerline[parent]
            child_cor = pointsCor3D_centerline[child]
            graph_centerline.add_edge(parent, child , weight=distance.euclidean(parent_cor, child_cor))
            childDepth = addBackChildren(child, curDepth + 1)
            
            if(childDepth < maxDepth):
                deleteChild(parent, child)
            else:
                maxDepth = childDepth
                
                if(curChild != -1):
                    deleteChild(curChild)
                    
                curChild = child           
        return maxDepth
     

In [21]:
def pointCorToVector(pointCor):
    x = pointCor[0]
    y = pointCor[1]
    z = pointCor[2]
    return vector(x, y, z)

In [22]:
# Uncomment the next line if want to reload from file
filePath = "mri_label_2016/*.png"
pointData = ReadPointFromPNG(filePath)

# the temp.npy stores the colon data points from 2016 model
#pointData = np.load('./temp.npy')
(graph, pointsCor3D) = getMSTFromDataPoint(pointData, drawMST=False, sampleNumber=5000)

---------------
Begin reading points data from PNG files
There are 144 PNG files, now convert them to point coordinates in 3D
Done!
---------------
There are 818757 points in total. Now sampleling 5000 points from them
---------------
Done!
---------------
Begin creating a MST of the sampled points cloud
---------------
Begin calculating nearby points for each point
---------------
Nearby points calculation Done!
---------------
Begin construct graph
---------------
Graph construction Done!
---------------
Begin calculate MST
---------------
MST calculation Done!
---------------
MST creation Done!


In [23]:
displayPoints(pointsCor3D, 1.3)

In [24]:
# the code to move points toward the centerline 
points_centerline = []
# The total number of iterations
for iteration in range(2):
    points_centerline = []
    H_ini = 15
    H_delta = 5
    H_glo = 0
    trial_limit =10
    min_neighbors = 150
    max_neighbors = 750
    correlation_threshold = 0.2 + iteration*0.05
    weight_dict = {}
    distance_dict = {}

    # Set up a dirty dictionary to record the points 
    dirty_dict = {}
    for point in range(len(pointsCor3D)):
        dirty_dict[point] = 0
    results = []
    
    # We will move 750 points of all the points for each iteration
    for targetPoint in range(3000, 3750):
        
        cur_correlation = 0
        H_glo = H_ini
        trial = 0
        correlation_hist = []
        localPointsCor2D_hist = []
        planeInfo = 0
        localPointsCor2D = []
        curPlaneGuess = [1, 1, 1]
        #curScalar = np.zeros(10)
        #curALen = 0
        
        # if the correlation of the local points is not sufficient or the number of the local points is not sufficient
        # enlarge H_glo and find neighbors again  
        while(cur_correlation < correlation_threshold and len(localPointsCor2D) < max_neighbors \
               and trial < trial_limit):
            
            curPlaneGuess = [1, 1, 1]
            localPointsCor2D, planeInfo = get2DCorFrom3D(targetPoint)
            
            slope, intersept = calculateRegressionLine(localPointsCor2D)
            rotatedPointsCor = rotatePointsCor(localPointsCor2D, slope)
            cur_correlation = abs(np.corrcoef(rotatedPointsCor[:, 0],rotatedPointsCor[:, 1])[0][1])
            print(targetPoint, "trial",str(trial), "H =", H_glo, ":" , str(cur_correlation), \
                  "size:", len(localPointsCor2D))
            H_glo += H_delta
            trial += 1
            correlation_hist.append(cur_correlation)
            localPointsCor2D_hist.append(copy.deepcopy(localPointsCor2D))
            
        localPointsCor2D = localPointsCor2D_hist[correlation_hist.index(max(correlation_hist))]
        centerPoint2D = np.asarray(localPointsCor2D)[:1, :2]
        newCor = Moving_Least_Square(centerPoint2D[0], np.asarray(localPointsCor2D)[:, :2])
        newCor3D = get3DCorFrom2D(newCor, planeInfo)
        print(pointsCor3D[targetPoint], newCor3D)
        pointsCor3D[targetPoint] = list(newCor3D)
        dirty_dict[targetPoint] = 1
        results.append(max(correlation_hist))
        points_centerline.append(newCor3D)
        
    displayPoints(points_centerline, 0.5)    

3000 trial 0 H = 30 : 0.048493400837464666 size: 161
3000 trial 1 H = 40 : 0.2365369799640109 size: 298
[202 131 163] [203.04521722 131.73734488 165.82640477]
3001 trial 0 H = 45 : 0.6163208956552527 size: 177
[142 420  85] [138.98071386 416.96293219  85.46031492]
3002 trial 0 H = 40 : 0.2130906834447351 size: 171
[186 170 159] [181.85084678 165.94440094 157.36034278]
3003 trial 0 H = 45 : 0.022468635416163008 size: 268
3003 trial 1 H = 55 : 0.029293889096833482 size: 315
3003 trial 2 H = 65 : 0.06060981475812923 size: 354
3003 trial 3 H = 75 : 0.4196382730045302 size: 414
[105 324 172] [ 95.69970441 312.95451445 168.16181634]
3004 trial 0 H = 35 : 0.5011584372840296 size: 161
[132 243 177] [133.67632809 244.19566497 180.36919052]
3005 trial 0 H = 45 : 0.3421309982066587 size: 238
[111 403  87] [110.59733013 405.15090727  99.6766441 ]
3006 trial 0 H = 35 : 0.028698510365631015 size: 165
3006 trial 1 H = 45 : 0.24275679775650041 size: 313
[308 131 133] [305.08009563 128.59160355 139.479

3050 trial 0 H = 40 : 0.0732261509412293 size: 158
3050 trial 1 H = 50 : 0.0031736140428231764 size: 188
3050 trial 2 H = 60 : 0.28846136635034764 size: 457
[257  72 160] [263.66004633  78.37084365 153.36478612]
3051 trial 0 H = 40 : 0.34261862209619826 size: 188
[217  65 147] [225.59763162  72.96266907 151.9604975 ]
3052 trial 0 H = 40 : 0.10777956068846853 size: 323
3052 trial 1 H = 50 : 0.012989815062773727 size: 667
3052 trial 2 H = 60 : 0.05050825189000569 size: 859
[185 103 154] [184.90454516 103.13522113 163.46823913]
3053 trial 0 H = 50 : 0.6579583849812567 size: 173
[130 395 163] [131.0111747  399.67129193 151.59913349]
3054 trial 0 H = 35 : 0.10933140847083789 size: 160
3054 trial 1 H = 45 : 0.056615635770911645 size: 283
3054 trial 2 H = 55 : 0.036832271700128545 size: 391
3054 trial 3 H = 65 : 0.004472620180076665 size: 508
3054 trial 4 H = 75 : 0.013508368226649574 size: 631
3054 trial 5 H = 85 : 0.010885258223928398 size: 1164
[157 145 175] [158.93868079 147.991669   170.

3103 trial 0 H = 35 : 0.04410398030979143 size: 163
3103 trial 1 H = 45 : 0.17973889328423745 size: 221
3103 trial 2 H = 55 : 0.575500338795675 size: 280
[105 304 180] [107.16218263 304.78893536 170.84044998]
3104 trial 0 H = 45 : 0.38495736594418856 size: 215
[147 391 115] [153.3624143  397.45124712 127.02919407]
3105 trial 0 H = 45 : 0.6473598953854915 size: 163
[219 432 141] [229.91639503 440.60559603 140.7695074 ]
3106 trial 0 H = 35 : 0.00219476750962823 size: 157
3106 trial 1 H = 45 : 0.039011311620206254 size: 228
3106 trial 2 H = 55 : 0.29413817208049986 size: 295
[ 90 331 157] [ 84.20657956 328.53094284 163.92369134]
3107 trial 0 H = 40 : 0.441697093417183 size: 154
[186 436  87] [176.63401777 428.86181261  91.00134339]
3108 trial 0 H = 45 : 0.5281827366106607 size: 163
[128 189 177] [132.30524989 193.71012449 182.75769125]
3109 trial 0 H = 40 : 0.024624377700629967 size: 160
3109 trial 1 H = 50 : 0.04598614501260502 size: 203
3109 trial 2 H = 60 : 0.009907535658799815 size: 3

3153 trial 2 H = 55 : 0.10112606170396654 size: 253
3153 trial 3 H = 65 : 0.43033796385398976 size: 343
[248 444 141] [246.62199442 443.13083526 143.20083533]
3154 trial 0 H = 35 : 0.07686574515302326 size: 159
3154 trial 1 H = 45 : 0.01599392317488192 size: 349
3154 trial 2 H = 55 : 0.009930690892407465 size: 489
3154 trial 3 H = 65 : 0.18283412067504543 size: 733
3154 trial 4 H = 75 : 0.10737715876555141 size: 939
[204  75 147] [212.83309349  91.33370018 158.75406078]
3155 trial 0 H = 45 : 0.4529301767194724 size: 221
[178 122 165] [181.87396134 123.55178125 173.49361326]
3156 trial 0 H = 35 : 0.08119683598374637 size: 156
3156 trial 1 H = 45 : 0.04570627843033488 size: 269
3156 trial 2 H = 55 : 0.04124344874512424 size: 337
3156 trial 3 H = 65 : 0.09066458566612552 size: 557
3156 trial 4 H = 75 : 0.09985597090763475 size: 647
3156 trial 5 H = 85 : 0.11384020678071338 size: 701
3156 trial 6 H = 95 : 0.08555894736265497 size: 844
[305 103 159] [301.98117583  99.8464303  146.69036263]


3198 trial 2 H = 60 : 0.15052196506716453 size: 789
[229  85 166] [226.89042182  92.99443452 160.34409647]
3199 trial 0 H = 40 : 0.2396046732361943 size: 164
[ 89 349 157] [ 83.00771699 344.15729896 159.87306751]
3200 trial 0 H = 40 : 0.1601543049487285 size: 159
3200 trial 1 H = 50 : 0.20693680551308116 size: 216
[ 90 398 123] [ 99.10100262 401.64521302 109.01560197]
3201 trial 0 H = 40 : 0.16752742932047573 size: 168
3201 trial 1 H = 50 : 0.1936266192741327 size: 336
3201 trial 2 H = 60 : 0.03671411497622818 size: 639
3201 trial 3 H = 70 : 0.007752986574370213 size: 855
[181  86 181] [187.06231905  91.77949626 164.30806294]
3202 trial 0 H = 50 : 0.6675496026533927 size: 153
[329 390 157] [325.9821803  386.99863683 148.28484426]
3203 trial 0 H = 50 : 0.09067665271657041 size: 215
3203 trial 1 H = 60 : 0.2560489985998826 size: 249
[ 64 320 159] [ 76.96547769 334.83529151 160.37007682]
3204 trial 0 H = 35 : 0.006566110234095337 size: 205
3204 trial 1 H = 45 : 0.22355760126199087 size: 3

...

[141 281 172] [140.67864221 280.70189488 172.99368146]
3543 trial 0 H = 55 : 0.565178083041378 size: 160
[381 240 153] [383.28355192 242.10969488 153.35766154]
3544 trial 0 H = 40 : 0.0033151928292495013 size: 175
3544 trial 1 H = 50 : 0.03910098437160337 size: 222
3544 trial 2 H = 60 : 0.2047245310872584 size: 268
3544 trial 3 H = 70 : 0.058471262984116076 size: 458
3544 trial 4 H = 80 : 0.022439085844030905 size: 627
3544 trial 5 H = 90 : 0.014957281104509336 size: 757
[176 176 154] [180.97517163 177.14454882 159.1318983 ]
3545 trial 0 H = 40 : 0.4365402100697458 size: 154
[ 77 351 155] [ 78.24325971 352.68103096 156.05951161]
3546 trial 0 H = 40 : 0.09928776723877658 size: 219
3546 trial 1 H = 50 : 0.3119844143847919 size: 400
[272  86 150] [277.23741286  90.92005331 151.34583683]
3547 trial 0 H = 60 : 0.497586542005862 size: 160
[383 234 153] [385.39442128 236.39880076 152.69239459]
3548 trial 0 H = 40 : 0.5904993942016963 size: 172
[179 422  98] [181.2776796  423.50683145  97.1105

3589 trial 0 H = 40 : 0.13558860192176028 size: 288
3589 trial 1 H = 50 : 0.24421256022630225 size: 375
3589 trial 2 H = 60 : 0.29194577408062333 size: 480
[298 132 138] [293.65956737 127.84313232 140.68371684]
3590 trial 0 H = 35 : 0.2541697142534232 size: 173
[152 150 171] [152.57659365 150.5989993  170.85448796]
3591 trial 0 H = 35 : 0.3411928127474688 size: 153
[129 240 180] [129.67495267 240.79217709 180.23567013]
3592 trial 0 H = 35 : 0.1986444781019412 size: 199
3592 trial 1 H = 45 : 0.011332036931761599 size: 414
3592 trial 2 H = 55 : 0.05277733609342616 size: 552
3592 trial 3 H = 65 : 0.08727699093640244 size: 679
3592 trial 4 H = 75 : 0.011503978185940384 size: 745
3592 trial 5 H = 85 : 0.08963180801333909 size: 862
[283 114 146] [287.66467321 119.04621485 142.76803543]
3593 trial 0 H = 45 : 0.48160093464138604 size: 198
[152 202 185] [142.22970316 193.7919768  184.9727149 ]
3594 trial 0 H = 35 : 0.31006957089932713 size: 197
[377 345 166] [376.10348284 344.51951707 168.17477

3632 trial 0 H = 35 : 0.03195712249312999 size: 206
3632 trial 1 H = 45 : 0.027715690464080196 size: 627
3632 trial 2 H = 55 : 0.03201554052792348 size: 844
[195  99 162] [197.62803186 101.42889945 162.65960968]
3633 trial 0 H = 45 : 0.650647534003258 size: 162
[330 371 147] [330.79951709 371.78533973 147.20671161]
3634 trial 0 H = 35 : 0.14294102191980496 size: 160
3634 trial 1 H = 45 : 0.06625654354882353 size: 237
3634 trial 2 H = 55 : 0.015697053002432335 size: 282
3634 trial 3 H = 65 : 0.029552083723002855 size: 498
3634 trial 4 H = 75 : 0.032046190868053825 size: 635
3634 trial 5 H = 85 : 0.02207838091714046 size: 749
3634 trial 6 H = 95 : 0.12936019544561536 size: 1259
[168 170 164] [179.65802617 181.24341706 167.16130325]
3635 trial 0 H = 35 : 0.5786004687956645 size: 169
[184  96 164] [182.21989588  93.54865699 167.16525391]
3636 trial 0 H = 35 : 0.36160663140940735 size: 206
[221 122 161] [217.74459797 119.14888424 163.75985714]
3637 trial 0 H = 50 : 0.7153685679726028 size: 

3675 trial 1 H = 45 : 0.2982497779675968 size: 246
[ 98 311 168] [ 97.70859966 311.07934991 169.42618169]
3676 trial 0 H = 45 : 0.11781332433534618 size: 163
3676 trial 1 H = 55 : 0.19506271014978632 size: 320
3676 trial 2 H = 65 : 0.14263612748799123 size: 431
3676 trial 3 H = 75 : 0.05814429192720621 size: 593
3676 trial 4 H = 85 : 0.034397397698178005 size: 697
3676 trial 5 H = 95 : 0.022001917781514006 size: 751
[276 149 136] [276.76947642 147.61841894 136.72810892]
3677 trial 0 H = 40 : 0.2985011549425317 size: 201
[224  77 154] [223.87011592  76.97042037 153.38398706]
3678 trial 0 H = 40 : 0.14756902610932157 size: 154
3678 trial 1 H = 50 : 0.18355878720776736 size: 280
3678 trial 2 H = 60 : 0.11737825379834765 size: 384
3678 trial 3 H = 70 : 0.09831571662669072 size: 537
3678 trial 4 H = 80 : 0.013642628737697867 size: 739
3678 trial 5 H = 90 : 0.028314294479500825 size: 810
[127 173 173] [126.26886467 171.46892097 174.54603478]
3679 trial 0 H = 35 : 0.0588922517301617 size: 249

3726 trial 1 H = 45 : 0.2893926586079352 size: 407
[279 104 149] [277.91473779 102.93652196 149.00068698]
3727 trial 0 H = 35 : 0.10581307385597911 size: 245
3727 trial 1 H = 45 : 0.1921694956772486 size: 310
3727 trial 2 H = 55 : 0.057129253421021486 size: 405
3727 trial 3 H = 65 : 0.09397216233361481 size: 433
3727 trial 4 H = 75 : 0.001291949636933913 size: 464
3727 trial 5 H = 85 : 0.3651719324375506 size: 514
[364 350 161] [363.92745265 350.26882336 162.2639145 ]
3728 trial 0 H = 40 : 0.0014827028020146852 size: 184
3728 trial 1 H = 50 : 0.0038754916780710425 size: 610
3728 trial 2 H = 60 : 0.04237211657149338 size: 907
[178 115 165] [182.26356989 118.96482811 166.73986762]
3729 trial 0 H = 35 : 0.26183341287349116 size: 195
[195 122 163] [198.67839488 123.68771249 168.23099592]
3730 trial 0 H = 50 : 0.5073726213375894 size: 169
[105 202 178] [109.70861185 206.93491268 179.58074611]
3731 trial 0 H = 35 : 0.35332406145241346 size: 170
[187 120 172] [187.38857355 119.90420045 173.71

In [25]:
# Construct a MST only using the 750 points modified above
#points_centerline = np.load('archive_npyfile/demoCenterline.npy')
(graph_centerline, pointsCor3D_centerline) = getMSTFromDataPoint(points_centerline, drawMST=True, sampleNumber=750)  

# constantly delete the node that only has one edge, until there are only two nodes only having one edge left,
# both of them are the endpoints of one singal path representing the colon

toRemove = []
removeCount = 0
removedNodeDict = defaultdict(list)

print("MST has", len(pointsCor3D_centerline), "nodes. Now begin to trim the graph.")

while (True):
    toRemove = []
    for node in graph_centerline.nodes():
        if(len(graph_centerline.edges(node)) == 1):
            removedNodeDict[list(graph_centerline.edges(node))[0][1]].append(node)
            toRemove.append(node)
    if(len(toRemove) == 2):
        break
    for node in toRemove:
        graph_centerline.remove_node(node)
        removeCount += 1
        toRemove = []
        
        
endpoints = toRemove
print("Done! Trimed", removeCount, "nodes. Now MST has", len(graph_centerline.nodes), "nodes left.")

print("Now begin reconstruct endpoints")
# now add back the nodes that got deleted during the triming
addBackChildren(endpoints[0], 0)
addBackChildren(endpoints[1], 0)

print("Done! Now MST has", len(graph_centerline.nodes), "nodes left.")
# Displat the points on the centerline
'''
to_display = []
for node in graph_centerline.nodes:
    to_display.append(pointsCor3D_centerline[node])
displayPoints(to_display, 1.3)
'''
# check if there is more than 2 endpoints
new_endpoints = []
for node in graph_centerline.nodes:
    if(len(graph_centerline.edges(node)) == 1):
       new_endpoints.append(node)
if(len(new_endpoints) != 2):
    print("Fatal error: multiple endpoints detected!")

# check if there is more than 2 path
path = list(nx.all_simple_paths(graph_centerline, source=new_endpoints[0], target=new_endpoints[1]))
if(len(path) != 1):
    print("Fatal error: multiple path detected!")
    
pointsInorder = path[0]

---------------
There are 750 points in total. Now sampleling 750 points from them
---------------
Done!
---------------
Begin creating a MST of the sampled points cloud
---------------
Begin calculating nearby points for each point
---------------
Nearby points calculation Done!
---------------
Begin construct graph
---------------
Graph construction Done!
---------------
Begin calculate MST
---------------
MST calculation Done!
---------------
MST creation Done!
MST has 750 nodes. Now begin to trim the graph.
Done! Trimed 409 nodes. Now MST has 341 nodes left.
Now begin reconstruct endpoints
Done! Now MST has 377 nodes left.


C:\Users\lizif\Anaconda3\envs\MIP3D\lib\site-packages\networkx\drawing\nx_pylab.py:611: MatplotlibDeprecationWarning: isinstance(..., numbers.Number)
  if cb.is_numlike(alpha):


In [26]:
c = curve()
for pointIndex in pointsInorder:
    c.append(pointCorToVector(pointsCor3D_centerline[pointIndex]))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
toDisplay=[]
for node in graph_centerline.nodes():
    toDisplay.append(pointsCor3D_centerline[node])
displayPoints(toDisplay, 1.3)